In [1]:
!pip install --upgrade azure-cognitiveservices-vision-contentmoderator

  Found existing installation: azure-cognitiveservices-vision-contentmoderator 0.1.0
    Uninstalling azure-cognitiveservices-vision-contentmoderator-0.1.0:
      Successfully uninstalled azure-cognitiveservices-vision-contentmoderator-0.1.0


In [6]:
from azure.cognitiveservices.vision.contentmoderator import ContentModeratorClient
import azure.cognitiveservices.vision.contentmoderator.models
from msrest.authentication import CognitiveServicesCredentials
from pprint import pprint

In [3]:
key = "144e9737e93647b3b93ac01219ac6855"
endpoint = "https://cmdemo01.cognitiveservices.azure.com/"

In [4]:
client = ContentModeratorClient(endpoint, CognitiveServicesCredentials(key))

### Text Moderation

In [10]:
with open("./Text Moderation.txt", "rb") as text_file:
    text_moderation = client.text_moderation.screen_text(text_content_type="text/plain", text_content=text_file, language="eng", autocorrect=True, pii=True)
    
    pprint(text_moderation.as_dict())

{'auto_corrected_text': 'This food is crap. \n'
                        '\n'
                        'My address is 1 Main St. New York, NY 10001 and my '
                        'phone number is 8645551111',
 'language': 'eng',
 'normalized_text': ' foood  crap. \n'
                    '\n'
                    ' address  1 Main St. New York, NY 10001   phone number  '
                    '8645551111',
 'original_text': 'This foood is crap. \n'
                  '\n'
                  'My address is 1 Main St. New York, NY 10001 and my phone '
                  'number is 8645551111',
 'pii': {'address': [{'index': 36, 'text': '1 Main St. New York, NY 10001'}],
         'email': [],
         'ipa': [],
         'phone': [{'country_code': 'US', 'index': 89, 'text': '8645551111'}],
         'ssn': []},
 'status': {'code': 3000, 'description': 'OK'},
 'terms': [{'index': 8, 'list_id': 0, 'original_index': 14, 'term': 'crap'}],
 'tracking_id': 'USE_ibiza_7b25ac66-7789-4f79-8332-7cb05601a3e

In [12]:
term_list = client.list_management_term_lists.create(
    content_type="application/json",
    body={
        "name": "Custom term list",
        "description": "Custom term description"
    }
)

In [13]:
client.list_management_term.add_term(list_id=term_list.id, term="shenanigans", language="eng")

{'ContentId': '545727',
 'AdditionalInfo': None,
 'Status': {'Code': 3000, 'Description': 'OK', 'Exception': None},
 'TrackingId': 'USE_ibiza_7b25ac66-7789-4f79-8332-7cb05601a3e8_ContentModerator.F0_643a8e54-2c47-416c-89df-4b3bb02c9bbc'}

In [16]:
custom_terms = client.list_management_term.get_all_terms(list_id=term_list.id, language="eng")

pprint(custom_terms.as_dict())

{'data': {'language': 'eng',
          'status': {'code': 3000, 'description': 'OK'},
          'terms': [{'term': 'shenanigans'}],
          'tracking_id': 'USE_ibiza_7b25ac66-7789-4f79-8332-7cb05601a3e8_ContentModerator.F0_5faac7eb-f518-4c7e-a630-d45933f86028'},
 'paging': {'limit': 50, 'offset': 0, 'returned': 1, 'total': 1}}


In [20]:
with open("./Custom Text.txt", "rb") as custom_text:
    screen_text = client.text_moderation.screen_text(
        text_content_type="text/plain", 
        text_content=custom_text, 
        language="eng", 
        autocorrect=True, 
        pii=True,
        list_id=term_list.id
    )
    
    pprint(screen_text.as_dict())

{'auto_corrected_text': 'The movie has a lot of shenanigans.',
 'language': 'eng',
 'normalized_text': ' movie   lot  shenanigans.',
 'original_text': 'The movie has a lot of shenanigans.',
 'status': {'code': 3000, 'description': 'OK'},
 'terms': [{'index': 14,
            'list_id': 1254,
            'original_index': 23,
            'term': 'shenanigans'}],
 'tracking_id': 'USE_ibiza_7b25ac66-7789-4f79-8332-7cb05601a3e8_ContentModerator.F0_af89c2f9-8ee9-4929-a542-81ad8c4b0b81'}


### Image Moderation

In [ ]:
# Family Photo by Jude Beck on Unsplash - https://unsplash.com/photos/a-nWU0o73r4



In [ ]:
# Swimsuit Photo by Amy Humphries on Unsplash - https://unsplash.com/photos/nvRUmQR3DFM

In [ ]:
# Billboard Photo by DesignClass on Unsplash - https://unsplash.com/photos/-c_LxZzVUnY

### Review